In [1]:
from llama_index.llms.openai import OpenAI
from llama_index.core.agent.workflow import AgentWorkflow
from llama_index.core.workflow import Context
from llama_index.core.agent.workflow import (
    AgentOutput,
    ToolCall,
    ToolCallResult,
)
from llama_index.tools.tavily_research import TavilyToolSpec
from llama_index.core.agent.workflow import FunctionAgent
import os
from llama_index.llms.ollama import Ollama


In [2]:
llm = OpenAI(model="gpt-3.5-turbo-1106", temperature=0)
#llm = Ollama("llama3.2:1b")


In [3]:
tavily_tool = TavilyToolSpec(api_key=os.getenv("TAVILY_API_KEY"))
search_web = tavily_tool.to_tool_list()[0]

In [4]:
search_web

In [5]:
async def record_notes(ctx: Context, notes: str, notes_title: str):
    current_state = await ctx.get("state")
    if "research_notes" not in current_state:
        current_state['research_notes'] = {}
    current_state['research_notes'][notes_title] = notes
    await ctx.set("state", current_state)
    return "Notes recorded"

In [6]:
async def write_report(ctx: Context, report_content:str):
    current_state = await ctx.get("state")
    current_state['report_content'] = report_content
    await ctx.set("state", current_state)
    return "Report written"

In [7]:
async def review_report(ctx: Context, review:str):
    current_state = await ctx.get("state")
    current_state['review'] = review
    await ctx.set("state", current_state)
    return "Report reviewed"

In [15]:
research_agent = FunctionAgent(
    name = "ResearchAgent",
    description = "USeful for searching the web for information",
    system_prompt = "You are a research assistant. You can search the web and record notes.",
    llm =llm,
    tools = [search_web, record_notes],
    can_handoff_to = ["WriteAgent"]
)

In [18]:
write_agent = FunctionAgent(
    name= "WriteAgent",
    description= "Useful for writing reports and reviewing them",
    system_prompt=(
        "You are the WriteAgent that can write a report on a given topic. "
        "Your report should be in a markdown format. The content should be grounded in the research notes. "
        "Once the report is written, you should get feedback at least once from the ReviewAgent."
    ),
    llm= llm,
    tools= [write_report],
    can_handoff_to= ["ReviewAgent", "ResearchAgent"]
)

In [19]:
review_agent = FunctionAgent(
    name= "ReviewAgent",
    description = "Useful for reviewing reports and providing feedback",
    tools = [review_report],
    llm = llm,
    system_prompt=(
        "You are the ReviewAgent that can review a report and provide feedback. "
        "Your feedback should either approve the current report or request changes for the WriteAgent to implement."
    ),
    can_handoff_to = ["WriteAgent"]
)

In [22]:
agent_workflow = AgentWorkflow(
    agents = [research_agent, write_agent, review_agent],
    root_agent = research_agent.name,
    initial_state = {
        "research_notes": {},
        "report_content": "Not written yet.",
        "review": "Review required"
    }
)

In [23]:
async def main():
    handler = agent_workflow.run(user_msg="""
        Write me a report on the history of the web. Briefly describe the history 
        of the world wide web, including the development of the internet and the 
        development of the web, including 21st century developments.
    """)

    current_agent = None
    current_tool_calls = ""
    async for event in handler.stream_events():
        if (
            hasattr(event, "current_agent_name")
            and event.current_agent_name != current_agent
        ):
            current_agent = event.current_agent_name
            print(f"\n{'='*50}")
            print(f"🤖 Agent: {current_agent}")
            print(f"{'='*50}\n")
        elif isinstance(event, AgentOutput):
            if event.response.content:
                print("📤 Output:", event.response.content)
            if event.tool_calls:
                print(
                    "🛠️  Planning to use tools:",
                    [call.tool_name for call in event.tool_calls],
                )
        elif isinstance(event, ToolCallResult):
            print(f"🔧 Tool Result ({event.tool_name}):")
            print(f"  Arguments: {event.tool_kwargs}")
            print(f"  Output: {event.tool_output}")
        elif isinstance(event, ToolCall):
            print(f"🔨 Calling Tool: {event.tool_name}")
            print(f"  With arguments: {event.tool_kwargs}")

if __name__ == "__main__":
    import asyncio
    asyncio.run(main())


🤖 Agent: ResearchAgent

🛠️  Planning to use tools: ['search', 'search', 'search']
🔨 Calling Tool: search
  With arguments: {'query': 'history of the internet'}
🔨 Calling Tool: search
  With arguments: {'query': 'development of the world wide web'}
🔨 Calling Tool: search
  With arguments: {'query': '21st century developments in web technology'}
🔧 Tool Result (search):
  Arguments: {'query': 'development of the world wide web'}
  Output: [Document(id_='f38b961f-8ce0-49aa-ad19-05cb6ab8d437', embedding=None, metadata={'url': 'https://www.britannica.com/topic/World-Wide-Web'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Tim Berners-Lee(more)\nThe development of the World Wide Web was begun in 1989 by Tim Berners-Lee and his colleagues at CERN, an international scientific organization based in Geneva, Switzerland. They created a pro

In [13]:
import nest_asyncio
nest_asyncio.apply()